## Esta notebook es creada para hacer pruebas con el procesamiento de la informacion de los documentos csv

In [74]:
import os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
import json
pd.options.mode.chained_assignment = None  # default='warn'

### _para limpiar los documentos extraidos del pdf_
    ->> obtener una lista de todas las materias con sus nrc y la carrera en la que se ofertan
    --- limpiar la lista de materias (toLower)
    --- sustituir los nombres de materias por los correctos
    ---reemplazar los NAN donde se genera esto

In [28]:
###celda de erorres TODO
data[0][16:22]

,Materia,Clave,Secc,NRC,Dias,Hora,Salon,Profesor,Doc
16,Inteligencia Artificial,CCOM 262,OO1,29089,AJ,0900-1059,1EMA4/409,ZACARIAS - FLORES FERNANDO,0
17,Inteligencia Artificial,CCOM 262,OO1,29089,L,1000-1059,1EMA4/409,NaN,0
18,Arqu. Funcional de,NaN,NaN,-1,NaN,NaN,NaN,ZACARIAS - FLORES FERNANDO,0
19,Computadora,IDCO 200,OO1,29093,L,1300-1359,1CCO5/201,NaN,0
20,Arqu. Funcional de,NaN,NaN,-1,NaN,NaN,NaN,SANCHEZ - GALVEZ MARIA EUGENIA N,0
21,Computadora,IDCO 200,OO1,29093,M,1300-1459,1CCO3/309,NaN,0


In [2]:
#recibe un directorio
#retorna todos los documentos de ese directorio
def getDocsinDir(directory):
    return list(map(lambda x: directory+x,os.listdir(directory)))
#recibe una lista de nombres de documentos
#retorna un lista de dataFrames
def getCSVs(docs):
    data=[]
    for x in docs:
        data.append(pd.read_csv(x))
    return data
#recibe un pandas.Dataframes
    #elimina todos los elementos nulos y ajusta los nombres de materias y profesores
#retorna el mismo Dataframe  
def cleanData(data):    
        data = data[data['Materia'].notna()]
        data['NRC']=data['NRC'].replace(to_replace=np.nan,value=-1)
        #data[2].dropna(inplace=True)
        data.reset_index(drop=True,inplace=True)
        bad=data.loc[lambda y: y.Materia=='Materia']
        #print(bad)
        badIndexes=bad.index.to_list()
        #print(badIndexes)
        data.reset_index(drop=True,inplace=True)
        data=data.drop(badIndexes)
        
        return data
#recibe el arreglo de dataframes 
##Recupera todas las materias con sus respectivos nrc
## devuelve un dataFrame de todas la materias con su nrc
def getCleanMaterias(data):
    
    materias=data[0].Materia.tolist()
    nrcs=data[0].NRC.tolist()
    numOfDoc=data[0].Doc.tolist()
    
    for x in range(len(data)-1):
        materias.extend(data[x+1].Materia.tolist())
        nrcs.extend(data[x+1].NRC.tolist())
        numOfDoc.extend(data[x+1].Doc.tolist())
        
    antesMat=""
    antesNrc=0
    realMaterias=[]
    for m,n,d in zip(materias,nrcs,numOfDoc):
        ahoraMat=m
        ahoraNrc=n
        if(antesNrc==-1):
            #print('aqui----'+antesMat+' '+m,n,type(n))
            helpDic={0:antesMat+' '+m,1:n,2:str(d)}
            realMaterias.append(helpDic)
        elif(ahoraNrc!=-1):
            helpDic={0:m,1:n,2:str(d)}
            #print(m,n,type(n))
            realMaterias.append(helpDic)
        antesNrc=n
        antesMat=m
    p=pd.DataFrame(realMaterias)
    p=p.rename(columns={0:'Materia',1:'NRC',2:'Doc'})
    p=p.drop_duplicates()
    return p

def get_materias(data):
    materias=data.Materia.tolist()
    nrcs=data.NRC.tolist()
    
    antesMat=""
    antesNrc=0
    realMaterias=[]
    for m,n in zip(materias,nrcs):
        ahoraMat=m
        ahoraNrc=n
        if(antesNrc==-1):
            #print('aqui----'+antesMat+' '+m,n,type(n))
            helpDic={0:antesMat+' '+m,1:n}
            realMaterias.append(helpDic)
        elif(ahoraNrc!=-1):
            helpDic={0:m,1:n}
            #print(m,n,type(n))
            realMaterias.append(helpDic)
        antesNrc=n
        antesMat=m
    p=pd.DataFrame(realMaterias)
    p=p.rename(columns={0:'Materia',1:'NRC'})
    p=p.drop_duplicates()
    return p

In [68]:
docs=[]
data=[]
Materias=[]
nombresMaterias=[]
def init(directorio):
    docs=getDocsinDir(directorio)
    data=getCSVs(docs)
    for x in range(len(data)):
            data[x]=cleanData(data[x])
            p=[x]*len(data[x])
            data[x]['Doc']=p ##genera una columna que almacene a que numero de documento pertenece
    Materias=getCleanMaterias(data)
    nombresMaterias=Materias.Materia.unique().tolist()
    return data
def get_Materias_por_plan():
    materias=[]
    for x in data:
        gg=get_materias(x)
        gg.reset_index(drop=True,inplace=True)
        none=[]
        for x in range(len(gg.Materia.unique().tolist())):
            none.append(None)
        pp=dict(zip(gg.Materia.unique().tolist(),none))
        materias.append(pp)
        ###materias.append(gg.Materia.unique().tolist())
        ###materias.append(gg)
    return materias

In [73]:
docs=getDocsinDir("app/filesCSV/")
data=init("app/filesCSV/")
ma=get_Materias_por_plan()


['app/filesCSV/COM-CUATRI-PRIMAVERA2020.csv',
 'app/filesCSV/COM-SEM-PRIMAVERA2020.csv',
 'app/filesCSV/ICC-CUIATRI-PRIMAVERA2020.csv',
 'app/filesCSV/ICC-SEM-PRIMAVERA2020.csv',
 'app/filesCSV/ITI-CUATRI-PRIMAVERA2020.csv',
 'app/filesCSV/ITI-SEM-PRIMAVERA2020.csv']

In [86]:
#def save_materias_json(dicts:list):
with open('ITI-6.json', 'w+') as fp:
    json.dump(ma[5],fp)

In [24]:
check_integrity()

In [76]:
data[0][["Materia","NRC"]].head()

,Materia,NRC
0,Lengua Extranjera III,20021
1,Lengua Extranjera IV,20022
2,Lengua Extranjera IV,20022
3,Analisis y Diseno de Algoritm,29066
4,Analisis y Diseno de Algoritm,29066


In [53]:
def check_integrity(cleaned,original_data):
    conta=0
    for x in cleaned["NRC"]:
        encontrado= False
        for y in original_data["NRC"]:
            if x == y :
                encontrado=True
                pass

        if not encontrado:
            print("no encontrado {}".format(x))
        else:
            conta+=1
    if len(cleaned) == conta:
        print("materias completas")
    else:
        print("error")

### _para indexador de palabras de documento de texto:_
        → retorna un diccionario con todas las palabras encontradas

In [5]:
#def simbologia
#???? => es necesario?
# [linea] ? [linea] => cual de las dos opciones es mejor?
#DEFINIR CLASE Match
#     class Match:
#      numerodelinea=int
#      ubicacion en la linea=[]  

#para indexador de indice invertido
    #leer todo el documento linea a linea
    #limpiar el texto
    #TOKENIZAR CADA PALABRA EN LA LINEA 
    #almacenar numero de palabras en la linea ????? (necesario si queremos calcular la pocision de la palabra relativa al doc)
    #si la palabra NO esta en stop words y aun no esta en el diccionario:
        #agregar la palabra al diccionario con su diccionario correspodiente
        #guardar en su diccionario el numero de documento al que corresponde
        #inicializar el primer elemnto de la lista (APARICIONES)=1 de la palabra en el diccionario del documento
        #crear una nueva instancia match
           #almacenar en que linea esta (NUM_LINEA)
            #almacenar la ubicacion de la palabra  (relativo a la linea)
                # almacenar que numero de palabra ocupa en la linea ? almacenar numero de caracter en el que inicia la palabra
        #agregar la instancia de Match a la lista del diccionario de la palabra
    #si la palabra ya esta en el diccionario
        #incrementar por uno (APARICIONES)
        #revisar el match.Linea anterior
            #si es la misma linea en la que está
                #actualizar la lista de apariciones
            #sino
                #crear una nueva instancia match
                #agregar la instancia de Match a la lista del diccionario de la palabra
        
#estructura: lista de diccionarios t.q:

#    {'token o palabra': {NUM_DOC:[APARICIONES,Match1,Match2],NUM_DOC:[APARICIONES,Match1,Match2]}}
# [[,,,,][,,,,]] cada lista corrsponde a  un documento y la pocision de la lista almacena el numero de palabras en la linea

In [82]:
class Match:
    def __init__(self,numeroLinea : int, ocurrencias: list):
        self.linea=numeroLinea
        self.ocurrencias=ocurrencias
    def addOcurrencias(self, new:list):
        for elem in new:
            if elem not in self.ocurrencias:
                self.ocurrencias.append(elem)
    def getOcurrencias(self):
        return self.ocurrencias
    def getLinea(self):
        return self.linea
    def getNumOcurrencias(self):
        return len(self.ocurrencias)

#clase modificada para que retorne una lista en lugar del objeto
class Match2():
    def __init__(self,match:list):
        self.linea=match[0]
        self.ocurrencias=match[1:]
    def getMatch(self):
        return [self.linea,self.ocurrencias]
    def addOcurrencias(self,new:list):
        for elem in new:
            if elem not in self.ocurrencias:
                self.ocurrencias.append(elem)
        return [self.linea,self.ocurrencias]
    def getOcurrencias(self):
        return self.ocurrencias[0]
    def getLinea(self):
        return self.linea
    def getNumOcurrencias(self,):
        return len(self.ocurrencias)
    

In [11]:
from nltk.tokenize import word_tokenize
import re
from unicodedata import normalize
stopWords=set(stopwords.words('spanish'))
stopWordsEngl=set(stopwords.words('english'))
#recibe una cadena
#retorna una cadena limpia
def normal(string):
    string=re.sub('_',' ',string)
    string=re.sub(r'[^\w\s]+',' ',string) #elimina simbolos
    string=re.sub(r"^\d+\s|\s\d+\s|\s\d+$",' ',string) ##elimina numeros solos
    string=re.sub(r"^\d+\s|\s\d+\s|\s\d+$",' ',string) ##elimina numeros solos
    string=string.lower()#↓↓ RETIRA DIACRITICOS MENOS ñ
   #string = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+",r"\1",normalize( "NFD",string),0,re.I)
    #string = normalize( 'NFC', string)
    return string

normal('ho4lá 5ño 1988´.dfs_?¡¿ 2 10921d aquíHOOL8 7545 :2012. 1901. más') #test

'ho4lá 5ño dfs   10921d aquíhool8    más'

In [27]:
#rutina para indexador
# recibe una lista de nombres de documentos 
#retorna un diccionario (la estructura del indexador)

def makeIndex(docs:list):
    headIndexer={}
    numeroPalabrasPorDoc=[]
    numDoc=0
    for x in docs:
        numeroPalabrasPorDoc.append([])
        tokens=[]
        doc=open(x,encoding='utf8')
        data=doc.readlines()
        ##para cada linea en el documento
        for i in range(len(data)):
            data[i]=normal(data[i])
            lineTokenized=word_tokenize(data[i])
            numeroPalabrasPorDoc[numDoc].append(len(lineTokenized))
            if len(lineTokenized) > 0:
                lineTokenized.insert(0,i) ##almacena el numero de linea al que pertenece en la cabeza de la lista
                tokens.append(lineTokenized)
    
        for linea in tokens:
            numpalabra=0
            numlinea=linea[0]
            for palabra in linea:
                if type(palabra) is not int:
                    if palabra not in stopWords and palabra not in stopWordsEngl:
                        if palabra not in headIndexer:
                            match=Match(numlinea,[numpalabra])
                            headIndexer.update({palabra:{numDoc:[1,match]}})## psible bug
                        else:
                            #si el documento no esta en elindexador
                            dicDocs= headIndexer.get(palabra)
                            if numDoc not in dicDocs:
                                match=Match(numlinea,[numpalabra])
                                dicDocs.update({numDoc:[1,match]})
                            #si el documento ya esta en el indexador
                            else:
                                headIndexer.get(palabra).get(numDoc)[0]+=1 ##actualiza a cantidad de ocurrencias
                                matchAnterior=headIndexer.get(palabra).get(numDoc)[-1]
                                if matchAnterior.getLinea() == numlinea:
                                    matchAnterior.addOcurrencias([numpalabra])
                                else:
                                    match=Match(numlinea,[numpalabra])
                                    headIndexer.get(palabra).get(numDoc).append(match)
                    numpalabra+=1
            numlinea+=1
        numDoc+=1
    return headIndexer

In [62]:
#rutina para indexador
# recibe una lista de nombres de documentos 
#retorna un diccionario (la estructura del indexador)

def makeIndex2(docs:list):
    headIndexer2={}
    numeroPalabrasPorDoc2=[]
    numDoc=0
    for x in docs:
        numeroPalabrasPorDoc2.append([])
        tokens=[]
        doc=open(x,encoding='utf8')
        data=doc.readlines()
        ##para cada linea en el documento
        for i in range(len(data)):
            data[i]=normal(data[i])
            lineTokenized=word_tokenize(data[i])
            numeroPalabrasPorDoc2[numDoc].append(len(lineTokenized))
            if len(lineTokenized) > 0:
                lineTokenized.insert(0,i) ##almacena el numero de linea al que pertenece en la cabeza de la lista
                tokens.append(lineTokenized)
    
        for linea in tokens:
            numpalabra=0
            numlinea=linea[0]
            for palabra in linea:
                if type(palabra) is not int:
                    if palabra not in stopWords and palabra not in stopWordsEngl:
                        if palabra not in headIndexer2:
                            match=Match2([numlinea,numpalabra]).getMatch()
                            headIndexer2.update({palabra:{numDoc:[1,match]}})## psible bug
                        else:
                            #si el documento no esta en elindexador
                            dicDocs= headIndexer2.get(palabra)
                            if numDoc not in dicDocs:
                                match=Match2([numlinea,numpalabra]).getMatch()
                                #print(match)
                                dicDocs.update({numDoc:[1,match]})
                            #si el documento ya esta en el indexador
                            else:
                                headIndexer2.get(palabra).get(numDoc)[0]+=1 ##actualiza la cantidad de ocurrencias
                                matchAnterior=headIndexer2.get(palabra).get(numDoc)[-1]
                                if matchAnterior[0] == numlinea:
                                    matchAnterior[1].append(numpalabra)
                                    #print(matchAnterior)
                                else:
                                    match=Match2([numlinea,numpalabra]).getMatch()
                                    headIndexer2.get(palabra).get(numDoc).append(match)
                    numpalabra+=1
            numlinea+=1
        numDoc+=1
    return headIndexer2

In [44]:
txt=["app/server.py","app/indexer.py"]

In [120]:
import json
ide=makeIndex(txt)
isinstance(ide.get("main")[0][1],Match)
class AEncoder(json.JSONEncoder):
    def default (self, obj):
        if type(obj)==Match:
            return obj.__dict__
        return json.JSONEncoder.default(self, obj)
    
with open('indexador.json', 'w+') as fp:
    json.dump(ide,fp,cls=AEncoder)

In [92]:
indexado=makeIndex2(txt)

In [35]:
#para buscar en el indexador
    # abrir todos los documentos y guaradar todos sus apuntadores en una lista
    # leer cada documento y en un array de lineas y alamacenarlos en una lista (hbuffer)
    #si la palabra esta en el indexador
        #recibir el diccionario de documentos correspondiente a la palabra
        #recorrer cada elemento en el diccionario
            #recorrer cada valor en la lista del elemento del diccionario
                #si el valor en la lista es un entero(el entero es el numero de incidencias)
                    #imprimir el numero ocurrencias y el nombre del documento
                #sino
                    #imprimir la lista de ocurrencias del primer match
                    #imprimir la linea del documento original almacenada en (hbuffer)
                    
            

In [49]:
hbuffer=[]
for x in range(len(txt)):
    hbuffer.append(None)
#funcion que recibe la palabra y el indexador e imprime los rsultados de la busqueda
def searchIndex(word,index):
    if word in index:
        dicDoc=index.get(word)
        for doc in dicDoc:
            #buffer.insert(doc,open(txt[doc],encoding='utf8'))
            #print(buffer)
            hbuffer[doc]=open(txt[doc],encoding='utf8').readlines()
            for value in dicDoc[doc]:
                if type(value) is int:
                    print("{} ocurrencias en {}".format(value,txt[doc]))
                else:
                    lst=list(map(lambda x: x+1,value.getOcurrencias())) #le suma uno a cada elemento de las ocurrencias
                    print("\tEn linea {} \n\t\tpalabra {}".format(value.getLinea()+1,lst))
                    print("\n\t{}".format(hbuffer[doc][value.getLinea()]))
    else:
        return None

In [85]:
hbuffer2=[]
for x in range(len(txt)):
    hbuffer2.append(None)
#funcion que recibe la palabra y el indexador e imprime los rsultados de la busqueda
def searchIndex2(word,index):
    if word in index:
        dicDoc=index.get(word)
        for doc in dicDoc:
            #buffer.insert(doc,open(txt[doc],encoding='utf8'))
            #print(buffer)
            hbuffer2[doc]=open(txt[doc],encoding='utf8').readlines()
            for value in dicDoc[doc]:
                if type(value) is int:
                    print("{} ocurrencias en {}".format(value,txt[doc]))
                else:
                    extractor=Match2(value)
                    #print(extractor.getOcurrencias())
                    #print(extractor.getMatch())
                    lst=list(map(lambda x: x+1,extractor.getOcurrencias())) #le suma uno a cada elemento de las ocurrencias
                    print("\tEn linea {} \n\t\tpalabra {}".format(extractor.getLinea()+1,lst))
                    print("\n\t{}".format(hbuffer2[doc][extractor.getLinea()]))
    else:
        return None

In [86]:
searchIndex2("main",indexado)

3 ocurrencias en app/server.py
	En linea 17 
		palabra [1]

	main

	En linea 29 
		palabra [3, 4]

	if __name__=='__main__':   main

1 ocurrencias en app/indexer.py
	En linea 98 
		palabra [3]

	if __name__ == '__main__':



In [87]:
searchIndex("main",ide)

3 ocurrencias en app/server.py
	En linea 17 
		palabra [1]

	

	En linea 29 
		palabra [3, 4]

	if __name__=='__main__':

1 ocurrencias en app/indexer.py
	En linea 98 
		palabra [3]

	if __name__ == '__main__':

